In [1]:
import logging
import random
import re
from collections import namedtuple
import Levenshtein as lev


### 1. Ajoutez d'autres règles au script pour rendre le chatbot encore plus convaincant.

#### Modifications faites dans le fichier doctor.txt :

* ajout de "Can you be more specific?" dans xnone
    * Cela permet de rendre le bot plus convaincant lorsqu'il ne comprend pas ce que l'on dit.

* ajout de "I am (3) when you are, what's happened?" dans decomp: * i am * @sad *
    * Cela permet de rendre le bot plus réel en incluant le mot "sad" et posant ensuite une question.
    
* Et ajout :    
   * key: computer 50
   *   decomp: *
   *     reasmb: Do computers worry you ?
   *     reasmb: Why do you mention computers ?
   *     reasmb: What do you think machines have to do with your problem ?
   *     reasmb: Don't you think computers can help people ?
   *     reasmb: What about machines worrys you ?
   *     reasmb: What do you think about machines ?

#### Modifications faites dans le script : 

* ajout d'un random choice de la réponse dans la decomp (c'est à dire parmi les réponses possibles)
* Lorsqu'on répète un même terme, au lieu de passer juste à la phrase suivante, il prend de manière aléatoire la réponse, sans prendre la précédente en compte. 



    
    def _next_reasmb(self, decomp):
        index = decomp.next_reasmb_index
        result = decomp.reasmbs[index % len(decomp.reasmbs)]
        
        #mon ajout 
        n=random.choice(range(len(decomp.reasmbs)))
        if n!= index:
            decomp.next_reasmb_index = n
        else :
            decomp.next_reasmb_index = index + 1
                
        return result




### 2. Penser à un scénario de conversation par exemple, voyage, cuisine, musique, et ajouter des règles au script pour soutenir une conversation dans ce scénario.

#### ajout de music en key :

* key: music 50
  * decomp: *
    * reasmb: Do music makes you happy ?
    * reasmb: Why do you mention music ?
    * reasmb: What do you think music have to do with your problem ?
    * reasmb: Don't you think music can help people ?
    * reasmb: What about RAP ?
    * reasmb: What do you think about music ?
    
* key: RAP 50
  * decomp: *
    * reasmb: Do you like Snoop Dog ?
    * reasmb: Who is the god of rap for you ?
    * reasmb: Don't you think RAP can help people ?

    

#### Script :
**Ce que je voulais faire mais que je n'ai pas eu le temps :**
* Lorsque le bot dit la phrase "What about RAP ?", une fois qu'on a répondu, il passe à la decomp de "RAP" par exemple en tombant sur "Do you like snoop dog ? ".
* Cela permet de maintenir une conversation sur le sujet de la musique et de la tourner vers le RAP qui en est une sous catégorie.
L'idée du script serait quelque chose du genre :

    
    if self.word == "music" && decomp_index == 4:
            key devient 'RAP'


### 3. What is your name? ne se comporte pas comme What is your name ? Modifiez le code python pour éviter ce problème.

#### On constate dans le fichier doctor.txt :

* key: name 15
  * decomp: *
    * reasmb: I am not interested in names.
    * reasmb: I've told you before, I don't care about names -- please continue.


#### On doit donc modifier le point d'intérogation comme un espace + point d'interrogation de cette manière :
        
        text = re.sub(r'\s*\?+\s*', ' ? ', text)


### 4. Modifiez le code pour qu'Eliza se rende compte si l'utilisateur répète les mêmes phrases et gérez correctement cette situation. Par exemple qu'Eliza dise "Stop repeting yourself". Utilisez la distance de Levenshtein pour comparer deux phrases ( exemple : what is your name? What's your name ? your name ? , se ressemblent)


#### L’algorithme de Levenshtein nous renvoit deux métriques : la distance et le ratio.
* #### Distance : 
    * C’est la mesure du nombre minimum de modifications (insertions, suppressions ou substitutions) qu’il faut effectuer pour changer une séquence d’un mot en l’autre.
* #### Ratio : 
    * Similarité entre deux mots

#### Nous utiliserons le ratio et dénissons que s'il est supérieur à 0.6 sur la string entiere, alors on renvoie Stop repeting yourself.

In [4]:
def levCalculate(str1, str2):
    Distance = lev.distance(str1, str2)
    Ratio = lev.ratio(str1, str2)
    print("Levenshtein entre {0} et {1}".format(str1, str2))
    print("> Distance: {0}\n> Ratio: {1}\n".format(Distance, Ratio))

levCalculate("What's your name?", "your name?")
levCalculate("What's your name?", "What is your name ?")

Levenshtein entre What's your name? et your name?
> Distance: 7
> Ratio: 0.7407407407407408

Levenshtein entre What's your name? et What is your name ?
> Distance: 3
> Ratio: 0.8888888888888888



#### L'implémentation est la suivante :

      def run(self):
        print(self.initial())
        input_text=["last","actual"]
        
        while True:
            sent = input('> ')
            input_text.append(sent)
            if levCalculate(input_text[-1], input_text[-2]) > 0.7 :
                output= "Stop repeting yourself !"
            else:
                output = self.respond(sent)
                if output is None:
                    break
            print(output)

        print(self.final())


**Je stock les inputs dans une liste et compare les derniers et avant derniers input pour obtenir leur distance de Levenshtein.** 
Le choix d'un ratio de Levenshtein à 0.7 est choisi après quelques tests sur des phrases.

Voici le rendu lorsqu'on lance le script (en gras les réponses d'Eliza) : 

* **How do you do.  Please tell me your problem.**
*  I am happy
* **How have I helped you to be happy ?**
*  I am happy
* **Stop repeting yourself !**
*  OK How are you ? 
* **Why do you ask ?**
*  OK How are you ?
* **Stop repeting yourself !**
*  What is your name ? 
* **I am not interested in names.**
*  your name ?
* **Stop repeting yourself !**
*  bye
* **Goodbye.  Thank you for talking to me.**  


### 5. Remplacez les mot clés dans (Clé, Patron, Règle de transformation) par des regexp.




### 6. Modifiez les script pour exploiter les regexp.


## Le code final 

### Remarques : 
* **Le fait d'avoir mit en place la mesure de Levenshtein est bénéfiques pour rendre Eliza plus convaincant. Cependant, cela implique que pour tester certaines questions précédentes cela est limitée** 
    * Par exemple, pour tester que la question 3 fonctionne, c'est à dire que "What is your name?" obtient le même schéma de décomposition que "What is your name ?", il ne faut pas poser les questions l'une après l'autre, leur distance de Levenshtein étant élevé. On aura donc comme résultat "Stop repeting yourself"**
    * Il en est de même si on ne dit pas correctement au revoir (par exemple "bye !" au lieu de "bye") pour arrêter le script, si on redis au revoir, alors cela dira "stop repeting yourself". Il faudra donc mettre "quit" par exemple, ou parler une fois d'autre chose avant de dire bye.**
* **Enfin, je n'ai pas réussi les deux dernières questions car je n'ai pas compris ce qu'il fallait réaliser.**

In [12]:
# Fix Python2/Python3 incompatibility
try: input = raw_input
except NameError: pass

log = logging.getLogger(__name__)


class Key:
    def __init__(self, word, weight, decomps):
        self.word = word
        self.weight = weight
        self.decomps = decomps

class Decomp:
    def __init__(self, parts, save, reasmbs):
        self.parts = parts
        self.save = save
        self.reasmbs = reasmbs
        self.next_reasmb_index = 0


class Eliza:
    def __init__(self):
        self.initials = []
        self.finals = []
        self.quits = []
        self.pres = {}
        self.posts = {}
        self.synons = {}
        self.keys = {}
        self.memory = []

    def load(self, path):
        key = None
        decomp = None
        with open(path) as file:
            for line in file:
                if not line.strip():
                    continue
                tag, content = [part.strip() for part in line.split(':')]
                if tag == 'initial':
                    self.initials.append(content)
                elif tag == 'final':
                    self.finals.append(content)
                elif tag == 'quit':
                    self.quits.append(content)
                elif tag == 'pre':
                    parts = content.split(' ')
                    self.pres[parts[0]] = parts[1:]
                elif tag == 'post':
                    parts = content.split(' ')
                    self.posts[parts[0]] = parts[1:]
                elif tag == 'synon':
                    parts = content.split(' ')
                    self.synons[parts[0]] = parts
                elif tag == 'key':
                    parts = content.split(' ')
                    word = parts[0]
                    weight = int(parts[1]) if len(parts) > 1 else 1
                    key = Key(word, weight, [])
                    self.keys[word] = key
                elif tag == 'decomp':
                    parts = content.split(' ')
                    save = False
                    if parts[0] == '$':
                        save = True
                        parts = parts[1:]
                    decomp = Decomp(parts, save, [])
                    key.decomps.append(decomp)
                elif tag == 'reasmb':
                    parts = content.split(' ')
                    decomp.reasmbs.append(parts)

    def _match_decomp_r(self, parts, words, results):
        if not parts and not words:
            return True
        if not parts or (not words and parts != ['*']):
            return False
        if parts[0] == '*':
            for index in range(len(words), -1, -1):
                results.append(words[:index])
                if self._match_decomp_r(parts[1:], words[index:], results):
                    return True
                results.pop()
            return False
        elif parts[0].startswith('@'):
            root = parts[0][1:]
            if not root in self.synons:
                raise ValueError("Unknown synonym root {}".format(root))
            if not words[0].lower() in self.synons[root]:
                return False
            results.append([words[0]])
            return self._match_decomp_r(parts[1:], words[1:], results)
        elif parts[0].lower() != words[0].lower():
            return False
        else:
            return self._match_decomp_r(parts[1:], words[1:], results)

    def _match_decomp(self, parts, words):
        results = []
        if self._match_decomp_r(parts, words, results):
            return results
        return None

    
    def _next_reasmb(self, decomp):
        #on prend l'index de la reponse qui au début est 0
        index = decomp.next_reasmb_index
        #on utilise cet index pour récupérer la réponse dans decomp de la class Key
        result = decomp.reasmbs[index % len(decomp.reasmbs)]
        
        #j'ajoute un random choice de l'index pour avoir une réponse aléatoire à partir de la décomposition de réponse dispo
        #Lorsque l'index choisi est différent du précédent, alors on l'utilise pour la prochaine réponse provenant
        #De la meme décomposition
        #Cependant si l'index est le meme que précédement, alors on lui ajoute +1 (ce qui été déjà le cas initialement)
        n=random.choice(range(len(decomp.reasmbs)))
        if n!= index:
            decomp.next_reasmb_index = n
        else :
            decomp.next_reasmb_index = index + 1
            
        return result

    
    def _reassemble(self, reasmb, results):
        output = []
        for reword in reasmb:
            if not reword:
                continue
            if reword[0] == '(' and reword[-1] == ')':
                index = int(reword[1:-1])
                if index < 1 or index > len(results):
                    raise ValueError("Invalid result index {}".format(index))
                insert = results[index - 1]
                for punct in [',', '.', ';']:
                    if punct in insert:
                        insert = insert[:insert.index(punct)]
                output.extend(insert)
            else:
                output.append(reword)
        return output

    def _sub(self, words, sub):
        output = []
        for word in words:
            word_lower = word.lower()
            if word_lower in sub:
                output.extend(sub[word_lower])
            else:
                output.append(word)
        return output

    def _match_key(self, words, key):
        for decomp in key.decomps:
            results = self._match_decomp(decomp.parts, words)
            if results is None:
                log.debug('Decomp did not match: %s', decomp.parts)
                continue
            log.debug('Decomp matched: %s', decomp.parts)
            log.debug('Decomp results: %s', results)
            results = [self._sub(words, self.posts) for words in results]
            log.debug('Decomp results after posts: %s', results)
            reasmb = self._next_reasmb(decomp)
            log.debug('Using reassembly: %s', reasmb)
            if reasmb[0] == 'goto':
                goto_key = reasmb[1]
                if not goto_key in self.keys:
                    raise ValueError("Invalid goto key {}".format(goto_key))
                log.debug('Goto key: %s', goto_key)
                return self._match_key(words, self.keys[goto_key])
            output = self._reassemble(reasmb, results)
            if decomp.save:
                self.memory.append(output)
                log.debug('Saved to memory: %s', output)
                continue
            return output
        return None

    def respond(self, text):
        
        if text.lower() in self.quits:
            return None

        text = re.sub(r'\s*\.+\s*', ' . ', text)
        text = re.sub(r'\s*,+\s*', ' , ', text)
        text = re.sub(r'\s*;+\s*', ' ; ', text)
        
        #on modifie "?" en " ? "
        text = re.sub(r'\s*\?+\s*', ' ? ', text)
        
        #Pour faire pour tous les charactères spéciaux etc..
        text = ''.join([e for e in text if e.isalnum() or e.isspace()])

        log.debug('After punctuation cleanup: %s', text)

        words = [w for w in text.split(' ') if w]
        log.debug('Input: %s', words)

        words = self._sub(words, self.pres)
        log.debug('After pre-substitution: %s', words)

        keys = [self.keys[w.lower()] for w in words if w.lower() in self.keys]
        keys = sorted(keys, key=lambda k: -k.weight)
        log.debug('Sorted keys: %s', [(k.word, k.weight) for k in keys])

        output = None

        for key in keys:
            output = self._match_key(words, key)
            if output:
                log.debug('Output from key: %s', output)
                break
        if not output:
            if self.memory:
                index = random.randrange(len(self.memory))
                output = self.memory.pop(index)
                log.debug('Output from memory: %s', output)
            else:
                output = self._next_reasmb(self.keys['xnone'].decomps[0])
                log.debug('Output from xnone: %s', output)

        return " ".join(output)

    def initial(self):
        return random.choice(self.initials)

    def final(self):
        return random.choice(self.finals)
    

    def run(self):
        print(self.initial())
        input_text=["last","actual"]
        
        while True:
            sent = input('> ')
            input_text.append(sent)
            if levCalculate(input_text[-1], input_text[-2]) > 0.7 :
                output= "Stop repeting yourself !"
            else:
                output = self.respond(sent)
                if output is None:
                    break
            print(output)

        print(self.final())


        
# Ma fonction de calcul de distance de Levenstein
def levCalculate(str1, str2):
    Ratio = lev.ratio(str1, str2)
    return Ratio


def main():
    eliza = Eliza()
    eliza.load('doctor.txt')
    eliza.run()

if __name__ == '__main__':
    logging.basicConfig()
    main()


How do you do.  Please tell me your problem.
> what is your name?
I am not interested in names.
> What's your name?
Stop repeting yourself !
> I am sad 
I am sad when you are, what's happened?
> I want to play music
Do music makes you happy ?
> Yes! I love music!
What about RAP ?
> I love RAP, and I love computer !
Do computers worry you ?
> Not at all !
I'm not sure I understand you fully.
> I am not worry about computers
What do you think about machines ?
> bye
Goodbye.  Thank you for talking to me.
